In [4]:
import numpy as np
from numpy import fft

from genfft.fft import make_twiddle

In [42]:
N = 36
factors = [4, 9]

x = np.arange(N)
s = x.reshape(factors[::-1]).T

# Notw fft
y = fft.fft(s, axis=1)

# Twiddle
y *= make_twiddle(4, 9).conj()
z = fft.fft(y.T).T.flatten()

In [43]:
abs(fft.fft(x) - z).max()

2.3517349180080774e-06

In [58]:
from functools import reduce
import operator

In [150]:
def reduce_factors_back(f):
    return f[:-2] + [f[-2] * f[-1]]

In [151]:
f = [4, 4, 4]
N = reduce(operator.mul, f)

# x = np.arange(N)
x = np.random.normal(size=[N])
s = x.reshape(f[::-1]).transpose()

# Notw fft
y = fft.fft(s, axis=-1)

# Twiddle
y *= make_twiddle(*f[-2:]).conj()
y = fft.fft(y, axis=1)

# Twiddle
f = reduce_factors_back(f)
y = y.reshape(f) * make_twiddle(*f).conj()
z = fft.fft(y, axis=0).flatten()

In [152]:
abs(fft.fft(x) - z).max()

3.7810183237622525e-07

In [97]:
def reduce_factors(f):
    return [f[0] * f[1]] + f[2:]

In [147]:
f = [2, 4, 4, 4, 4]
N = reduce(operator.mul, f)

# x = np.arange(N)
x = np.random.normal(size=[N])
s = x.reshape(f).transpose()
p = np.arange(N).reshape(f).transpose()

# Notw fft
y = fft.fft(s, axis=-1)
print("notw_{}()".format(f[0]))

while len(f) > 1:
    # Twiddle
    y *= make_twiddle(*f[1::-1]).conj()
    y = fft.fft(y, axis=-2)
    print("twiddle_{}()".format(f[1]))
    f = reduce_factors(f)
    y = y.reshape(f[::-1])

notw_2()
twiddle_4()
twiddle_4()
twiddle_4()
twiddle_4()


In [140]:
N

512

In [141]:
abs(fft.fft(x) - y).max()

3.4571333006311086e-06

In [146]:
p[..., 0, 0].flatten() * 2

array([ 0, 16, 32, 48,  4, 20, 36, 52,  8, 24, 40, 56, 12, 28, 44, 60,  2,
       18, 34, 50,  6, 22, 38, 54, 10, 26, 42, 58, 14, 30, 46, 62])